In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pretrain-model-robert/pretrain_model_0901/config.json
/kaggle/input/pretrain-model-robert/pretrain_model_0901/pytorch_model.bin
/kaggle/input/pretrain-model-robert/pretrain_model_0901/vocab.txt
/kaggle/input/dialog-script/script/data_process.py
/kaggle/input/dialog-script/script/train.py
/kaggle/input/dialog-script/script/train_kaggle.py
/kaggle/input/dialog-script/script/bert_config_kaggle.py
/kaggle/input/dialog-script/script/test.py
/kaggle/input/dialog-script/script/data_process_org_train_without_span.py
/kaggle/input/dialog-script/script/bert_config.py
/kaggle/input/dialog-script/script/utils.py
/kaggle/input/dialog-script/script/requirements.txt
/kaggle/input/dialog-script/script/data_analysis.py
/kaggle/input/dialog-script/script/models.py
/kaggle/input/dialog-script/script/predict.py
/kaggle/input/dialog-script/script/load_data.py
/kaggle/input/dialog-house-train-data/train_data/span_first_label2id.json
/kaggle/input/dialog-house-train-data/train_data/span_df_trai

In [2]:
import os
print(os.getcwd())

/kaggle/working


In [3]:
import transformers
import torch

print(transformers.__version__)
print(torch.__version__)

4.20.1
1.11.0


In [4]:
# !pip uninstall transformers --yes
# !pip uninstall torch --yes
# !pip uninstall protobuf --yes

In [5]:
# !pip install protobuf==3.19.4 
# !pip install transformers==4.21.2
# !pip install torch==1.10.0


In [6]:
import sys
sys.path.append('/kaggle/input/dialog-script/script/')

In [7]:
import os
import logging
from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch
import torch.nn as nn
import numpy as np
from transformers import BertTokenizer
import  json

## ***bert config

In [8]:
import argparse
class Args:
    @staticmethod
    def parse():
        parser = argparse.ArgumentParser()
        return parser
    @staticmethod
    def initialize(parser):
        # args for path
        parser.add_argument('--output_dir', default='./checkpoints/',help='the output dir for model checkpoints')
        #parser.add_argument('--bert_dir', default='/kaggle/input/pretrain-model-robert/pretrain_model_0901/', help='bert dir for uer')
        parser.add_argument('--bert_dir', default='hfl/chinese-roberta-wwm-ext', help='bert dir for uer')
        parser.add_argument('--data_dir', default='/kaggle/input/dialog-house-train-data/train_data/', help='data dir for uer')
        parser.add_argument('--log_dir', default='./logs/', help='log dir for uer')
        # other args
        parser.add_argument('--num_tags', default=65, type=int,help='number of tags')
        parser.add_argument('--seed', type=int, default=123, help='random seed')

        parser.add_argument('--gpu_ids', type=str, default="0,1",help='gpu ids to use, -1 for cpu, "0,1" for multi gpu')
        parser.add_argument('--max_seq_len', default=256, type=int)
        parser.add_argument('--eval_batch_size', default=1, type=int)
        parser.add_argument('--swa_start', default=3, type=int, help='the epoch when swa start')
        # train args
        parser.add_argument('--train_epochs', default=1, type=int,help='Max training epoch')
        parser.add_argument('--dropout_prob', default=0.2, type=float,help='drop out probability')
        # 2e-5
        parser.add_argument('--lr', default=3e-5, type=float,help='learning rate for the bert module')
        # 2e-3
        parser.add_argument('--other_lr', default=3e-4, type=float,help='learning rate for the module except bert')
        # 0.5
        parser.add_argument('--max_grad_norm', default=1, type=float,help='max grad clip')
        parser.add_argument('--warmup_proportion', default=0.1, type=float)
        parser.add_argument('--weight_decay', default=0.01, type=float)
        parser.add_argument('--adam_epsilon', default=1e-8, type=float)
        parser.add_argument('--train_batch_size', default=1, type=int)
        parser.add_argument('--eval_model', default=True, action='store_true',help='whether to eval model after training')
        return parser

    def get_parser(self):
        parser = self.parse()
        parser = self.initialize(parser)
        return parser.parse_args(args=[])
    
logger = logging.getLogger(__name__)
args =Args()
args = args.get_parser()
print(args)

Namespace(adam_epsilon=1e-08, bert_dir='hfl/chinese-roberta-wwm-ext', data_dir='/kaggle/input/dialog-house-train-data/train_data/', dropout_prob=0.2, eval_batch_size=1, eval_model=True, gpu_ids='0,1', log_dir='./logs/', lr=3e-05, max_grad_norm=1, max_seq_len=256, num_tags=65, other_lr=0.0003, output_dir='./checkpoints/', seed=123, swa_start=3, train_batch_size=1, train_epochs=1, warmup_proportion=0.1, weight_decay=0.01)


# data loader

In [9]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

# Torch数据集导入类
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len,class_type):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['text']
        self.first_label = dataframe['fir_key']
        self.targets = self.data.target_list
        self.max_len = max_len
        self.class_type = class_type

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        if self.class_type == 'first':
            text = str(self.text[index])
            first_label = None
        else:
            text = str(self.text[index])
            first_label = str(self.first_label[index])
        inputs = self.tokenizer.encode_plus(
            text,
            first_label,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation='longest_first'
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {'token_ids': torch.tensor(ids, dtype=torch.long),
                'attention_masks': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'labels': torch.tensor(self.targets[index], dtype=torch.float)
                }


def data_loader(raw_csv_path,class_type):
    df_raw = pd.read_csv(raw_csv_path)
    if class_type == 'first':
        df_raw['target_list'] = df_raw['first_key_list_flag'].apply(lambda x: x.split(','))
    else:
        df_raw['target_list'] = df_raw['second_key_list_flag'].apply(lambda x: x.split(','))

    df_raw['target_list'] = df_raw['target_list'].apply(lambda x: [float(i) for i in x])
    df_raw = df_raw.head(50)
    print('---df columns ',df_raw.columns)
    print('-----df_raw before', df_raw.shape)
    df = df_raw[['fir_key', 'text', 'target_list']].copy()
    df.dropna(inplace=True)
    print('-----df_raw after', df.shape)
    df.to_csv('./test.txt', index=False)
    

    # 划分数据, 训练集:测试集=8:2
    train_size = 0.8
    train_dataset = df.sample(frac=train_size, random_state=200)
    valid_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)
    print(f"FULL Dataset: {df.shape}, "
          f"TRAIN Dataset: {train_dataset.shape}, "
          f"TEST Dataset: {valid_dataset.shape}")
    # 使用改造后的Torch DataLoader加载数据集
    tokenizer = BertTokenizer.from_pretrained(args.bert_dir)
    training_set = CustomDataset(train_dataset, tokenizer, args.max_seq_len, class_type)
    validation_set = CustomDataset(valid_dataset, tokenizer, args.max_seq_len, class_type)

    train_params = {'batch_size': args.train_batch_size, 'shuffle': False, 'num_workers': 0}
    test_params = {'batch_size': args.eval_batch_size, 'shuffle': False, 'num_workers': 0}

    training_loader = DataLoader(training_set, **train_params)
    validation_loader = DataLoader(validation_set, **test_params)
    return training_loader, validation_loader

def show_batch(loader):
    for step,sample in enumerate(loader): # 每一次挑选出来的size个数据
        print(list(sample.keys()))
        print('----token_ids', sample['token_ids'].shape)
        print('----token_ids', sample['attention_masks'].shape)
        print('----token_ids', sample['token_type_ids'].shape)
        print('----token_ids', sample['labels'].shape)


# utils

In [10]:
# coding=utf-8
import random
import logging
import time
import numpy as np
import torch


def timer(func):
    """
    函数计时器
    :param func:
    :return:
    """

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print("{}共耗时约{:.4f}秒".format(func.__name__, end - start))
        return res

    return wrapper


def set_seed(seed=123):
    """
    设置随机数种子，保证实验可重现
    :param seed:
    :return:
    """
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    #torch.cuda.manual_seed_all(seed)


def set_logger(log_path):
    """
    配置log
    :param log_path:s
    :return:
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 由于每调用一次set_logger函数，就会创建一个handler，会造成重复打印的问题，因此需要判断root logger中是否已有该handler
    if not any(handler.__class__ == logging.FileHandler for handler in logger.handlers):
        file_handler = logging.FileHandler(log_path)
        formatter = logging.Formatter(
            '%(asctime)s - %(levelname)s - %(filename)s - %(funcName)s - %(lineno)d - %(message)s')
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

    if not any(handler.__class__ == logging.StreamHandler for handler in logger.handlers):
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)


# model

In [11]:
from transformers import BertModel
import torch.nn as nn


class BertForMultiLabelClassification(nn.Module):
    def __init__(self, args):
        super(BertForMultiLabelClassification, self).__init__()
        self.bert = BertModel.from_pretrained(args.bert_dir)
        self.bert_config = self.bert.config
        out_dims = self.bert_config.hidden_size
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(out_dims, args.num_tags)

    def forward(self, token_ids, attention_masks, token_type_ids):
#         print('---input shape', token_ids.shape, attention_masks.shape, token_type_ids.shape)
#         print('---token_ids ', token_ids[0])
#         print('---attention_masks ', attention_masks[0])
#         print('---token_type_ids ', token_type_ids[0])
        
        bert_outputs = self.bert(
            input_ids = token_ids,
            attention_mask = attention_masks,
            token_type_ids = token_type_ids
        )
        print('----bert out', bert_outputs.pooler_output.shape)
        #seq_out = bert_outputs[1]
        seq_out = bert_outputs.pooler_output
        print('----seq_out', seq_out.shape)
        seq_out = self.dropout(seq_out)
        seq_out = self.linear(seq_out)
        return seq_out

In [12]:
import torch
 
# 返回当前设备索引
 
print(torch.cuda.current_device())
 
# 返回GPU的数量
print(torch.cuda.device_count())
 
# 返回gpu名字，设备索引默认从0开始
print(torch.cuda.get_device_name(0))
 
# cuda是否可用
print(torch.cuda.is_available())
 
# cuda当前的版本
print(torch.version.cuda)
 
# 测试程序
a = torch.Tensor(5,3)
a=a.cuda()
print(a)

0
1
Tesla P100-PCIE-16GB
True
11.0
tensor([[ 0.0000e+00,  0.0000e+00,  1.8077e-43],
        [ 0.0000e+00,  1.8669e+23,  3.0719e-41],
        [ 1.8719e+23,  3.0719e-41,  1.8618e+23],
        [ 3.0719e-41,  5.6052e-45,  0.0000e+00],
        [ 1.8604e+23,  3.0719e-41, -1.0208e-15]], device='cuda:0')


In [13]:
from transformers import BertTokenizer, BertModel
import torch
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

gpu_ids = args.gpu_ids.split(',')
device = torch.device("cpu" if gpu_ids[0] == '-1' else "cuda:" + gpu_ids[0])
print(device)
        

tokenizer = BertTokenizer.from_pretrained(args.bert_dir)
model = BertModel.from_pretrained(args.bert_dir)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
print(inputs)
#outputs = model(**inputs)

model = BertForMultiLabelClassification(args)
outputs = model(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'])

#last_hidden_states = outputs.last_hidden_state
print(outputs.shape)

cuda:0


Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[  101,  8701,   117,  8422, 13030,  8310, 12322,  8154,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
torch.Size([1, 65])


In [14]:
class Trainer:
    def __init__(self, args, train_loader, dev_loader, test_loader, device):
        self.args = args
        gpu_ids = args.gpu_ids.split(',')
        self.device = torch.device("cpu" if gpu_ids[0] == '-1' else "cuda:" + gpu_ids[0])
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = BertForMultiLabelClassification(args)
        self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr=self.args.lr)
        self.criterion = nn.BCEWithLogitsLoss()
        self.train_loader = train_loader
        self.dev_loader = dev_loader
        self.test_loader = test_loader
        self.model.to(self.device)

    def load_ckp(self, model, optimizer, checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        return model, optimizer, epoch, loss

    def save_ckp(self, state, checkpoint_path):
        torch.save(state, checkpoint_path)

    """
    def save_ckp(self, state, is_best, checkpoint_path, best_model_path):
        tmp_checkpoint_path = checkpoint_path
        torch.save(state, tmp_checkpoint_path)
        if is_best:
            tmp_best_model_path = best_model_path
            shutil.copyfile(tmp_checkpoint_path, tmp_best_model_path)
    """

    def train(self):
        print('----self device', self.device)
        total_step = len(self.train_loader) * self.args.train_epochs
        global_step = 0
        eval_step = 1 # 决定每多少步，保存一次模型
        best_dev_micro_f1 = 0.0
        for epoch in range(args.train_epochs):
            for train_step, train_data in enumerate(self.train_loader):
                self.model.train()
#                 print('---input shape', train_data['token_ids'].shape, train_data['attention_masks'].shape, train_data['token_type_ids'].shape)
#                 print('---token_ids ', train_data['token_ids'][0])
#                 print('---attention_masks ', train_data['attention_masks'][0])
#                 print('---token_type_ids ', train_data['token_type_ids'][0])
                token_ids = train_data['token_ids'].to(self.device)
                attention_masks = train_data['attention_masks'].to(self.device)
                token_type_ids = train_data['token_type_ids'].to(self.device)
                labels = train_data['labels'].to(self.device)
                try:
                    train_outputs = self.model(token_ids, attention_masks, token_type_ids)
                except:
                    print('------except  have data nan')
                    continue
                print('-----bert model output')
                loss = self.criterion(train_outputs, labels)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                logger.info(
                    "【train】 epoch：{} step:{}/{} loss：{:.6f}".format(epoch, global_step, total_step, loss.item()))
                global_step += 1
                if global_step % eval_step == 0:
                    dev_loss, dev_outputs, dev_targets = self.dev()
                    accuracy, micro_f1, macro_f1 = self.get_metrics(dev_outputs, dev_targets)
                    logger.info(
                        "【dev】 loss：{:.6f} accuracy：{:.4f} micro_f1：{:.4f} macro_f1：{:.4f}".format(dev_loss, accuracy,
                                                                                                   micro_f1, macro_f1))
                    if macro_f1 > best_dev_micro_f1:
                        logger.info("------------>保存当前最好的模型")
                        checkpoint = {
                            'epoch': epoch,
                            'loss': dev_loss,
                            'state_dict': self.model.state_dict(),
                            'optimizer': self.optimizer.state_dict(),
                        }
                        best_dev_micro_f1 = macro_f1
                        checkpoint_path = os.path.join(self.args.output_dir, 'best.pt')
                        print('-----save checkpoint_path', checkpoint_path)
                        self.save_ckp(checkpoint, checkpoint_path)

    def dev(self):
        self.model.eval()
        total_loss = 0.0
        dev_outputs = []
        dev_targets = []
        with torch.no_grad():
            for dev_step, dev_data in enumerate(self.dev_loader):
                token_ids = dev_data['token_ids'].to(self.device)
                attention_masks = dev_data['attention_masks'].to(self.device)
                token_type_ids = dev_data['token_type_ids'].to(self.device)
                labels = dev_data['labels'].to(self.device)
                outputs = self.model(token_ids, attention_masks, token_type_ids)
                loss = self.criterion(outputs, labels)
                # val_loss = val_loss + ((1 / (dev_step + 1))) * (loss.item() - val_loss)
                total_loss += loss.item()
                outputs = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
                outputs = (np.array(outputs) > 0.6).astype(int)
                dev_outputs.extend(outputs.tolist())
                dev_targets.extend(labels.cpu().detach().numpy().tolist())

        return total_loss, dev_outputs, dev_targets

    def test(self, checkpoint_path):
        model = self.model
        optimizer = self.optimizer
        model, optimizer, epoch, loss = self.load_ckp(model, optimizer, checkpoint_path)
        model.eval()
        model.to(self.device)
        total_loss = 0.0
        test_outputs = []
        test_targets = []
        with torch.no_grad():
            for test_step, test_data in enumerate(self.test_loader):
                token_ids = test_data['token_ids'].to(self.device)
                attention_masks = test_data['attention_masks'].to(self.device)
                token_type_ids = test_data['token_type_ids'].to(self.device)
                labels = test_data['labels'].to(self.device)
                outputs = model(token_ids, attention_masks, token_type_ids)
                loss = self.criterion(outputs, labels)
                # val_loss = val_loss + ((1 / (dev_step + 1))) * (loss.item() - val_loss)
                total_loss += loss.item()
                outputs = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
                outputs = (np.array(outputs) > 0.6).astype(int)
                test_outputs.extend(outputs.tolist())
                test_targets.extend(labels.cpu().detach().numpy().tolist())

        return total_loss, test_outputs, test_targets

    def predict(self, tokenizer, text, id2label, args):
        model = self.model
        optimizer = self.optimizer
        checkpoint = os.path.join(args.output_dir, 'best.pt')
        model, optimizer, epoch, loss = self.load_ckp(model, optimizer, checkpoint)
        model.eval()
        model.to(self.device)
        with torch.no_grad():
            inputs = tokenizer.encode_plus(text=text,
                                        add_special_tokens=True,
                                        max_length=args.max_seq_len,
                                        truncation='longest_first',
                                        padding="max_length",
                                        return_token_type_ids=True,
                                        return_attention_mask=True,
                                        return_tensors='pt')
            token_ids = inputs['input_ids'].to(self.device)
            attention_masks = inputs['attention_mask'].to(self.device)
            token_type_ids = inputs['token_type_ids'].to(self.device)
            outputs = model(token_ids, attention_masks, token_type_ids)
            outputs = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
            outputs = (np.array(outputs) > 0.5).astype(int)
            outputs = np.where(outputs[0] == 1)[0].tolist()
            if len(outputs) != 0:
                #print('----output',outputs )
                outputs = [id2label[str(i)] for i in outputs]
                return outputs
            else:
                return '不好意思，我没有识别出来'

    def get_metrics(self, outputs, targets):
        accuracy = accuracy_score(targets, outputs)
        micro_f1 = f1_score(targets, outputs, average='micro')
        macro_f1 = f1_score(targets, outputs, average='macro')
        return accuracy, micro_f1, macro_f1

    def get_classification_report(self, outputs, targets, labels):
        # confusion_matrix = multilabel_confusion_matrix(targets, outputs)
        report = classification_report(targets, outputs, target_names=labels)
        return report


In [15]:
def main():
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    if not os.path.exists('./checkpoints/'):
        os.makedirs('./checkpoints/')

    if not os.path.exists('./logs/'):
        os.makedirs('./logs/')


    class_type = 'first'
    print('---当前目录',os.getcwd())
    #args = bert_config.Args().get_parser()
    print('----args')
    #set_seed(args.seed)
    set_logger(os.path.join(args.log_dir, 'main.log'))
    set_seed(args.seed)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('-----device',device)

    # load label 映射id
    if class_type == 'first':
        label2id = json.load(open(args.data_dir + '/span_first_label2id.json'))
        train_file_name = 'span_df_train_mulitclass_data.txt'
    else:
        label2id = json.load(open(args.data_dir + '/span_second_label2id.json'))
        train_file_name = 'span_df_train_mulitclass_data.txt'

    id2label = {}
    labels = []
    for key, value in label2id.items():
        id2label[value] = key
        labels.append(key)
    args.num_tags=len(labels)
    print('------id2label',id2label)

    # 加载数据集
    train_loader, dev_loader = data_loader(args.data_dir + train_file_name,class_type)
    #print('--train_loader', train_loader)
    
    #show_batch(train_loader)
    

    trainer = Trainer(args, train_loader, dev_loader, dev_loader,device)
    #训练和验证
    trainer.train()
    
    
#     # 测试
#     logger.info('========进行测试========')
#     #checkpoint_path = args.output_dir + 'best.pt'
#     checkpoint_path = os.path.join(args.output_dir, 'best.pt')
#     print('-----test load path',checkpoint_path)
#     total_loss, test_outputs, test_targets = trainer.test(checkpoint_path)
#     accuracy, micro_f1, macro_f1 = trainer.get_metrics(test_outputs, test_targets)
#     logger.info("【test】 loss：{:.6f} accuracy：{:.4f} micro_f1：{:.4f} macro_f1：{:.4f}".format(total_loss, accuracy, micro_f1, macro_f1))
#     report = trainer.get_classification_report(test_outputs, test_targets, labels)
#     logger.info(report)
    
#     # 预测
#     trainer = Trainer(args, None, None, None)
#     #checkpoint_path = './checkpoints/best.pt'
#     checkpoint_path = os.path.join(args.output_dir, 'best.pt')
#     tokenizer = BertTokenizer.from_pretrained(args.bert_dir)
    
#     # 读取test1.json里面的数据, 一级预测， 二级预测
#     import pandas as pd
#     df_test = pd.read_csv(args.data_dir + '/span_df_train_mulitclass_data.txt')
    
#     df_test = df_test.head(10)
#     test_text = df_test['text'].values.tolist()
#     all_result = []
#     for t in test_text:
#         result = trainer.predict(tokenizer, t, id2label, args)
#         all_result.append(result)
#         print('-----text',t)
#         print(result)
#     # 预测单条
#     text = '8岁男童海螺沟失联13日，父母悬赏10万寻子，马上就到他9岁生日了'
#     print(trainer.predict(tokenizer, text, id2label, args))


In [16]:
main()

---当前目录 /kaggle/working
----args
-----device cuda
------id2label {'0': '关注点', '1': '置业意向', '2': '意向产品', '3': '置业预算', '4': '贷款方式', '5': '意向户型', '6': '意向购房面积', '7': '家庭结构', '8': '首付情况', '9': '置业次数'}
---df columns  Index(['first_key_list', 'second_key_list', 'text', 'first_key_list_flag',
       'second_key_list_flag', 'file_id', 'fir_key', 'sec_key', 'target_list'],
      dtype='object')
-----df_raw before (50, 9)
-----df_raw after (50, 3)
FULL Dataset: (50, 3), TRAIN Dataset: (40, 3), TEST Dataset: (10, 3)


Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


----self device cuda


【train】 epoch：0 step:0/40 loss：0.624422


----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
-----bert model output
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])


...

----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
-----bert model output
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
【dev】 loss：3.038189 accuracy：0.0000 micro_f1：0.0000 macro_f1：0.0000
【train】 epoch：0 step:39/40 loss：0.501282


----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
-----bert model output
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
【dev】 loss：3.012737 accuracy：0.0000 micro_f1：0.0000 macro_f1：0.0000


----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
----bert out torch.Size([1, 768])
----seq_out torch.Size([1, 768])
